<a href="https://colab.research.google.com/github/AruneshDev/Analyzing-Historical-Stock-Revenue-Data-and-Building-a-Dashboard/blob/main/World_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install streamlit ngrok


### Fetching a live stock data

In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px

# Define top 50 tech stock tickers
top_tech_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "ADBE", "NFLX", "CRM",
    "AMD", "INTC", "ORCL", "IBM", "CSCO", "UBER", "PYPL", "SHOP", "SQ", "SNOW",
    "ASML", "AVGO", "TXN", "QCOM", "NOW", "ZM", "TWLO", "PLTR", "DOCU", "WDAY",
    "TEAM", "FSLY", "CRWD", "OKTA", "DDOG", "NET", "PINS", "SNAP", "ROKU", "SPOT",
    "ZS", "MDB", "VEEV", "PANW", "FIS", "ADSK", "SAP", "INTU", "ANET", "SE"
]

# Function to get live stock data and market cap
def get_stock_data(tickers):
    data = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1d", interval="5m")  # 5-minute interval

            if hist.empty:
                print(f"Warning: No recent data available for {ticker}, skipping...")
                continue

            latest = hist.iloc[-1]  # Get the most recent data

            data.append({
                "Ticker": ticker,
                "Price": latest["Close"],
                "Change (%)": ((latest["Close"] - latest["Open"]) / latest["Open"]) * 100,
                "Market Cap": info.get("marketCap", 0)  # Get market cap (0 if unavailable)
            })
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(data)

# Fetch stock data
df = get_stock_data(top_tech_tickers)

# Convert Market Cap to Billions
df["Market Cap (B)"] = df["Market Cap"] / 1e9  # Convert to billions

# Display DataFrame
print(df)


Error fetching data for SQ: list index out of range
   Ticker        Price  Change (%)     Market Cap  Market Cap (B)
0    AAPL   245.589996   -0.117947  3688676720640     3688.676721
1    MSFT   408.140015   -0.115017  3034624819200     3034.624819
2   GOOGL   179.660004    0.038982  2200601362432     2200.601362
3    AMZN   216.490005   -0.018470  2295249764352     2295.249764
4    TSLA   337.540009   -0.062174  1086540414976     1086.540415
5    NVDA   134.270004   -0.533374  3292190474240     3292.190474
6    META   683.640015   -0.118343  1731883171840     1731.883172
7    ADBE   444.230011   -0.036005   193412497408      193.412497
8    NFLX  1002.489990   -0.060810   429104431104      429.104431
9     CRM   309.779999    0.096933   296478572544      296.478573
10    AMD   110.790001   -0.261074   179613990912      179.613991
11   INTC    24.850000   -0.679457   107687100416      107.687100
12   ORCL   167.869995   -0.136830   469357854720      469.357855
13    IBM   261.579987  

### Stock Real-Time Heatmap

In [2]:

# Create a Heatmap with Market Cap as Size
fig = px.treemap(df,
                 path=["Ticker"],
                 values="Market Cap (B)",  # Market Cap as size
                 color="Change (%)",  # Color based on percentage change
                 color_continuous_scale=["red", "white", "green"],
                 title="Live Tech Stock Heatmap (Sized by Market Cap)")

fig.show()


### Install Libraries

In [ ]:
!pip install yfinance ipywidgets

### Investment Calculator

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Define top 50 tech stocks
top_tech_stocks = {
    "Apple (AAPL)": "AAPL", "Microsoft (MSFT)": "MSFT", "Google (GOOGL)": "GOOGL", "Amazon (AMZN)": "AMZN",
    "Tesla (TSLA)": "TSLA", "Nvidia (NVDA)": "NVDA", "Meta (META)": "META", "Adobe (ADBE)": "ADBE",
    "Netflix (NFLX)": "NFLX", "Salesforce (CRM)": "CRM", "AMD": "AMD", "Intel (INTC)": "INTC",
    "Oracle (ORCL)": "ORCL", "IBM": "IBM", "Cisco (CSCO)": "CSCO", "Uber (UBER)": "UBER",
    "Paypal (PYPL)": "PYPL", "Shopify (SHOP)": "SHOP", "Square (SQ)": "SQ", "Snowflake (SNOW)": "SNOW",
    "ASML": "ASML", "Broadcom (AVGO)": "AVGO", "Texas Instruments (TXN)": "TXN", "Qualcomm (QCOM)": "QCOM",
    "ServiceNow (NOW)": "NOW", "Zoom (ZM)": "ZM", "Twilio (TWLO)": "TWLO", "Palantir (PLTR)": "PLTR",
    "DocuSign (DOCU)": "DOCU", "Workday (WDAY)": "WDAY", "Atlassian (TEAM)": "TEAM", "Fastly (FSLY)": "FSLY",
    "CrowdStrike (CRWD)": "CRWD", "Okta (OKTA)": "OKTA", "Datadog (DDOG)": "DDOG", "Cloudflare (NET)": "NET",
    "Pinterest (PINS)": "PINS", "Snap (SNAP)": "SNAP", "Roku (ROKU)": "ROKU", "Spotify (SPOT)": "SPOT",
    "Zscaler (ZS)": "ZS", "MongoDB (MDB)": "MDB", "Veeva (VEEV)": "VEEV", "Palo Alto Networks (PANW)": "PANW",
    "Fiserv (FIS)": "FIS", "Autodesk (ADSK)": "ADSK", "SAP": "SAP", "Intuit (INTU)": "INTU",
    "Arista Networks (ANET)": "ANET", "Sea Limited (SE)": "SE"
}

# Function to calculate compound returns
def calculate_returns(amount, years, annual_return):
    return amount * (1 + annual_return / 100) ** years

# Function to fetch stock's past 5-year CAGR
def get_cagr(stock_ticker):
    try:
        stock = yf.Ticker(stock_ticker)
        hist = stock.history(period="5y")  # Fetch last 5 years of data

        if hist.empty:
            return 10  # Default CAGR if data is missing

        start_price = hist["Close"].iloc[0]
        end_price = hist["Close"].iloc[-1]

        # Calculate CAGR: ((End Price / Start Price)^(1/years)) - 1
        cagr = ((end_price / start_price) ** (1 / 5) - 1) * 100
        return round(cagr, 2)
    except:
        return 10  # Default to 10% CAGR if fetch fails

# Create interactive widgets
stock_dropdown = widgets.Dropdown(
    options=top_tech_stocks.keys(),
    description="Stock:"
)

amount_input = widgets.IntText(
    value=1000,
    min=100,
    step=100,
    description="Investment ($):"
)

years_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=30,
    step=1,
    description="Years:"
)

# Button to calculate
calc_button = widgets.Button(description="Calculate Returns", button_style='success')

# Output widget
output = widgets.Output()

# Function to update investment growth plot
def update_investment(change):
    with output:
        output.clear_output()

        # Get user input
        selected_stock = stock_dropdown.value
        selected_ticker = top_tech_stocks[selected_stock]
        amount = amount_input.value
        years = years_slider.value

        # Fetch CAGR
        expected_return = get_cagr(selected_ticker)

        # Calculate projected return
        final_value = calculate_returns(amount, years, expected_return)

        # Display results
        print(f"📊 **Stock:** {selected_stock} ({selected_ticker})")
        print(f"📈 **Historical Growth Rate:** {expected_return}% CAGR")
        print(f"💰 **Investment:** ${amount:,} for {years} years")
        print(f"💸 **Projected Return:** **${final_value:,.2f}**")

        # Investment Growth Chart
        x = np.arange(0, years + 1)
        y = [calculate_returns(amount, i, expected_return) for i in x]

        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(x, y, marker="o", linestyle="-", color="green", label="Investment Growth")
        ax.set_title(f"Investment Growth Over Time for {selected_ticker}")
        ax.set_xlabel("Years")
        ax.set_ylabel("Total Value ($)")
        ax.legend()
        ax.grid(True)
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'${x:,.0f}'))

        plt.show()

# Attach function to button click
calc_button.on_click(update_investment)

# Display UI
display(stock_dropdown, amount_input, years_slider, calc_button, output)


Dropdown(description='Stock:', options=('Apple (AAPL)', 'Microsoft (MSFT)', 'Google (GOOGL)', 'Amazon (AMZN)',…

IntText(value=1000, description='Investment ($):', step=100)

IntSlider(value=5, description='Years:', max=30, min=1)

Button(button_style='success', description='Calculate Returns', style=ButtonStyle())

Output()